In [1]:
import pandas as pd
import glob
import numpy as np
from sklearn.neighbors import BallTree, KDTree, DistanceMetric
import lightgbm as lgb
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import subprocess
from scipy.spatial import distance

from tqdm import tqdm_notebook as tqdm

from tqdm import tnrange as trange

In [2]:
test_data = pd.read_csv('data/dm5/test_data13_changed_50.csv.zip')
print(test_data.shape)
test_data.head()

(2958072, 7)


,id,TX,TY,X,Y,Z,data_ind
0,0,-0.243285,0.034970,49282.421875,49359.332031,40083.0,0
1,1,-0.111049,0.052493,41839.175781,21390.554688,33618.0,0
2,2,-0.047433,-0.269513,60404.886719,28631.410156,42669.0,0
3,3,-0.081808,0.022675,31875.050781,30950.125000,62064.0,0
4,4,0.386942,-0.271280,36173.644531,25627.705078,36204.0,0


In [3]:
test_example = pd.read_csv('data/dm5/test_example13_changed_50.csv.zip')
print(test_example.shape)
test_example.head()

(2958072, 51)


,id,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,...,class_40,class_41,class_42,class_43,class_44,class_45,class_46,class_47,class_48,class_49
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train = pd.DataFrame()
i = 0
for f in tqdm(glob.glob('data/dm5/open/open*.h5')):
    chunk = pd.read_hdf(f)
    train = train.append(chunk, ignore_index=True)
    i += 1
    print(f)
print(train.shape)
train.head()

Widget Javascript not detected.  It may not be installed or enabled properly.


data/dm5/open/open30.h5
data/dm5/open/open0.h5
data/dm5/open/open40.h5
data/dm5/open/open10.h5
data/dm5/open/open70.h5
data/dm5/open/open80.h5
data/dm5/open/open50.h5
data/dm5/open/open20.h5
data/dm5/open/open60.h5

(120421126, 8)


,TX,TY,X,Y,Z,event_id,signal,data_ind
0,-0.043787,0.149312,37343.636719,57321.039062,25860.0,-999.0,0.0,21
1,-0.238449,-0.369275,38199.117188,14818.970703,72408.0,-999.0,0.0,21
2,0.312426,-0.253441,36330.500000,31390.966797,69822.0,-999.0,0.0,21
3,0.028330,-0.237798,43192.093750,46829.191406,18102.0,-999.0,0.0,21
4,0.318769,-0.355580,44290.164062,71907.625000,18102.0,-999.0,0.0,21


In [5]:
for data_ind in tqdm(test_data.data_ind.unique()):
    brick = test_data.loc[test_data['data_ind'] == data_ind]
    print(brick.loc[np.ceil(brick['Z']/1293) != brick['Z']/1293].shape[0])

Widget Javascript not detected.  It may not be installed or enabled properly.


88
85



In [6]:
for data_ind in tqdm(train.data_ind.unique()):
    brick = train.loc[train['data_ind'] == data_ind]
    print(brick.event_id.nunique() - 1 - brick.loc[np.ceil(brick['Z']/1293) != brick['Z']/1293].shape[0])

Widget Javascript not detected.  It may not be installed or enabled properly.


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0



In [ ]:
while 0:
    for data_ind in tqdm(train.data_ind.unique()):
        brick = train.loc[train['data_ind'] == data_ind]
        brick_signal = brick.loc[brick['signal'] == 1]
        for event_id in brick_signal['event_id'].unique():
            event = brick.loc[brick['event_id'] == event_id]
            frac_index = event.loc[np.ceil(event['Z']/1293) != event['Z']/1293].index[0]
            origin_index = event.Z.idxmin()
            if (frac_index != origin_index):
                if (not(event.loc[frac_index].Z - 1293 < event.loc[origin_index].Z <= event.loc[frac_index].Z)):
                    origin = event.loc[origin_index]
                    frac = event.loc[frac_index]
                    if (event.loc[origin_index].event_id != event.loc[frac_index].event_id):
                        print('200')
                    #print(event.loc[frac_index].Z- event.loc[origin_index].Z)
                    print('------')
                    print(event.loc[origin_index].TX, event.loc[origin_index].TY,
                          event.loc[origin_index].X, event.loc[origin_index].Y, event.loc[origin_index].Z)
                    print(event.loc[frac_index].TX, event.loc[frac_index].TY,
                          event.loc[frac_index].X, event.loc[frac_index].Y, event.loc[frac_index].Z)
                    fig = plt.figure()
                    ax = fig.gca(projection='3d')
                    ax.view_init(elev=10, azim=-85)    
                    ax.quiver(event['Z'], event['Y'], event['X'], 1293, 1293*event['TY'], 1293*event['TX'],
                              arrow_length_ratio=0, linewidths=0.7)
                    plt.show()
    break

In [ ]:
real_train_data_list = []

for data_ind in tqdm(train.data_ind.unique()):
    brick = train.loc[train['data_ind'] == data_ind]
    brick_signal = brick.loc[brick['signal'] == 1]
    real_train_data_temp = pd.DataFrame()
    for event_id in tqdm(brick_signal['event_id'].unique(),leave=False):
        event = brick.loc[brick['event_id'] == event_id]
        frac_index = event.loc[np.ceil(event['Z']/1293) != event['Z']/1293].index[0]
        # Добавка
        f = event.loc[frac_index][['TX','TY','X','Y','Z']].values.reshape(1,5)
        # Добавляем сигнальные треки
        new_brick = pd.DataFrame()
        new_brick[['TX','TY','X','Y','Z']] = event[['TX','TY','X','Y','Z']].reset_index(drop=True)
        new_brick[['DTX','DTY','DX','DY','DZ']] = pd.DataFrame(new_brick.values-f)
        new_brick['S'] = 1
        real_train_data_temp = real_train_data_temp.append(new_brick, ignore_index=True)
        # Добавляем не сигнальные треки
        not_event = brick_signal.loc[brick['event_id'] != event_id].sample(n=event.shape[0])
        new_brick = pd.DataFrame()
        new_brick[['TX','TY','X','Y','Z']] = not_event[['TX','TY','X','Y','Z']].reset_index(drop=True)
        new_brick[['DTX','DTY','DX','DY','DZ']] = pd.DataFrame(new_brick.values-f)
        new_brick['S'] = 0
        real_train_data_temp = real_train_data_temp.append(new_brick, ignore_index=True)
    real_train_data_list.append(real_train_data_temp)

real_train_data = pd.concat(real_train_data_list, ignore_index=True)
real_train_data_list = []

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


...

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [ ]:
print(real_train_data.shape)
real_train_data.head()

In [ ]:
X_train = real_train_data.drop(['S'],axis=1)
y_train = real_train_data['S']
lgb_train = lgb.Dataset(X_train, y_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.03,
    'max_depth': 15,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 4,
    'early_stopping_round': 100
}

In [ ]:
num_round = 10
lgb.cv(params, lgb_train, num_round, nfold=3)

In [ ]:
num_round = 4000
bst = lgb.train(params, lgb_train, num_round)

In [ ]:
cols = ['id'] + ['class_%d' % (i) for i in range(100)]
res_pred = pd.DataFrame(0, index=np.arange(test_data.shape[0]), columns=cols)
res_pred['id'] = res_pred['id'].index
print(res_pred.shape)
res_pred.head()

In [ ]:
for data_ind in tqdm(test_data.data_ind.unique()):
    brick = test_data.loc[test_data['data_ind'] == data_ind]
    fracs = brick.loc[np.ceil(brick['Z']/1293) != brick['Z']/1293]
    # ищем центр кирпича
    center = ((brick[['X','Y','Z']].min() + brick[['X','Y','Z']].max()) / 2).values.tolist()
    dists = []
    for i in range(fracs.shape[0]):
        # distance
        dist = distance.euclidean(fracs.iloc[i][['X','Y','Z']].values.tolist(), center)
        dists.append(dist)
    fracs.insert(fracs.columns.shape[0],'DISTS', dists)
    fracs = fracs.sort_values('DISTS')
    for i in tqdm(range(fracs.shape[0]),leave=False):
        # Добавка
        f = fracs.iloc[i][['TX','TY','X','Y','Z']].values
        # Добавляем сигнальные треки
        new_brick = pd.DataFrame()
        new_brick[['TX','TY','X','Y','Z']] = brick[['TX','TY','X','Y','Z']].reset_index(drop=True)
        new_brick[['DTX','DTY','DX','DY','DZ']] = pd.DataFrame(new_brick.values-f)
        # классифицируем
        ypred = bst.predict(new_brick)
        res_pred.loc[brick['id'],'class_%d' % (i+1)] = ypred
        res_pred.loc[brick['id'],'class_0'] += (1 - ypred) / fracs.shape[0]

In [23]:
for data_ind in tqdm(test_data.data_ind.unique()):
    brick = test_data.loc[test_data['data_ind'] == data_ind]
    fracs = brick.loc[np.ceil(brick['Z']/1293) != brick['Z']/1293]
    res_pred.loc[brick['id'],'class_0'] = 1 - res_pred.loc[:,'class_1':'class_99'].sum(axis=1) / fracs.shape[0]

In [24]:
res_pred_50 = res_pred.loc[:,'id':'class_49']
print(res_pred_50.shape)
res_pred_50.head()

(2958072, 51)


,id,class_0,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,...,class_40,class_41,class_42,class_43,class_44,class_45,class_46,class_47,class_48,class_49
0,0,0.952682,5.915180e-07,2.881952e-06,0.002092,9.574932e-01,0.000087,2.224064e-11,6.150174e-05,3.022469e-67,...,2.877712e-21,2.279555e-16,1.494452e-19,3.784262e-25,0.000014,1.478341e-18,2.054059e-04,4.372139e-04,3.475600e-03,3.778448e-52
1,1,0.992670,5.065678e-06,2.469770e-10,0.000008,1.474520e-07,0.000015,7.791927e-12,1.742771e-08,9.795850e-09,...,1.979419e-15,3.355900e-08,2.211401e-04,7.215956e-12,0.005317,7.897169e-49,5.775081e-81,1.883314e-20,7.571001e-17,1.138275e-09
2,2,0.985115,5.147873e-06,6.688174e-05,0.000033,3.356106e-04,0.000003,6.338171e-02,1.996580e-05,5.575104e-06,...,1.431211e-05,1.749606e-03,5.015541e-07,4.152475e-07,0.000501,3.316892e-58,1.241312e-03,5.894975e-123,5.370899e-06,1.072215e-03
3,3,0.944560,9.531241e-05,1.673217e-04,0.000269,1.391569e-04,0.001063,1.527854e-06,8.593282e-04,2.742755e-06,...,1.014487e-09,3.983288e-06,5.320639e-03,1.267416e-06,0.025800,5.895632e-33,2.734471e-07,2.851174e-06,6.580263e-04,1.115936e-06
4,4,0.955457,4.121226e-06,1.014579e-07,0.000015,2.495934e-06,0.000025,1.535638e-51,5.653034e-07,5.087832e-08,...,1.159048e-129,4.224819e-08,3.348050e-04,4.776805e-11,0.069378,8.727850e-113,2.839436e-35,5.492553e-18,2.451626e-42,1.881957e-08


In [25]:
res_pred_50.to_csv('data/dm5/output.csv.gz', float_format='%.3f', compression='gzip', index=False)

In [27]:
1

1